# Python 教程 - 一些高级特性
## 核心：函数即对象
类不仅可以有实例方法，也可以有静态方法。
调用类方法时不需要传入实例（因而也没有self参数）

In [1]:
class normal_class:
    def static_method(name):
        print(name + 'nb!')
    def instance_method(self, name):
        print(name + 'nb!')

a = normal_class()
a.instance_method('tyh')
normal_class.static_method('sqc')

tyhnb!
sqcnb!


既然方法和函数行为如此相似，能不能把它们统一起来？
能！通过可调用对象！

In [2]:
class might_be_a_function:
    def __call__(self):
        print('hello, class!')

def function():
    print('hello, function!')

i = might_be_a_function()
i()
function()

hello, class!
hello, function!


在Python中，函数只不过是一个可调用的对象。（准确来说是带类方法的__call__，但不关键（

In [3]:
print(type(function))
print(isinstance(function, object))

<class 'function'>
True


既然函数也是对象，自然可以像使用对象一样使用他。
具体来说，函数对象可以作为参数(函数式编程，lambda表达式)，也可以作为返回值（闭包，装饰器）；

# 函数对象作为参数 - 函数式编程与lambda表达式
对列表做一些变换生成新的列表是一个常见的任务，这有着大量的应用。
有时，我们希望筛选列表中的一些值，生成一个新列表。
又有时，我们的目标是遍历列表的每一个值，将他们按某种方式“加起来”，返回一个值。
这两个操作能构筑非常丰富的组件，实际上，在分布式系统中，这种编程模式（map-reduce），是最方便快捷的方式。

当然，这些任务都能用循环实现，但对比较简单的变换，这种实现比较繁琐，另外，循环只能处理有限值。
函数式编程为我们提供了一种更简单的实现列表数据的变换方式，并且有能力处理循环处理不了的问题。

In [4]:
a = [1,2,3,4,5]
result = [] # 如果a是偶数，值取 True；否则取 False
for k in a:
    if k % 2 == 0:
        result.append(True)
    else:
        result.append(False)
print(result)

[False, True, False, True, False]


In [11]:
def is_even(x):
    return x % 2 == 0

result = map(is_even, a) # map 的第一个参数是一个函数！
print(result)# 是一个迭代器，可以作为 for 循环的迭代对象，但无法直接获取值！
print(list(result))

[False, True, False, True, False]


## 能否更短？
这种简单的函数，很可能只用一次，又非常简单，能否有一种“一次性”的函数，让他直接嵌入语句中？

有！lambda函数（匿名函数）为我们提供了有力的工具。
Python的lambda函数形式为lambda parameterList: body

In [10]:
result = map(lambda x: x % 2 == 0, a)
print(list(result))
result = map(lambda x: 1 if x % 2 == 0 else 0, a) # 一个常见的技巧
print(list(result))

[False, True, False, True, False]
[0, 1, 0, 1, 0]


## 什么时候用 lambda 表达式？
lambda表达式带来简洁性的同时带来的是易读性的下降，
在Python中，请**绝对不要**滥用 lambda 表达式，只对满足这些条件的函数使用 lambda 表达式：
- 只用一两次
- 非常简单

只要不满足上面的任何一条，就单独写一个函数。
另外，永远不要使用嵌套 lambda 表达式。

In [12]:
result = filter(lambda x: x % 2 == 0, a) # filter，传入一个返回True或者False的函数和一个可迭代对象，构建一个列表
# 其中的值为使函数返回 True 的那些值
print(result)
print(list(result))

[2, 4]


In [13]:
import functools
result = functools.reduce(lambda s, x: s + x**2, a) # reduce，传入一个带两个参数的函数和一个可迭代对象，构建一个值
# 函数的第一个参数是前面所有元素运算得到的结果，第二个参数是当前值。
# 如果 reduce 没有传入第三个参数，初始 s 为0，否则为第三个参数的值。
print(result)

55


# 函数对象作为返回值 - 闭包与装饰器
## 闭包
如同前面所提到的，函数也可以作为函数、方法的返回值。

In [14]:
def wrapper():
    def get_sum(x, y):
        return x + y
    return get_sum # 不带括号，直接返回函数对象！

sum_func = wrapper() # wrapper 返回了一个函数，现在 sum_func 是一个函数对象
print(sum_func(1, 2))

3


可是，这有什么用呢？
使用闭包，我们让一个内部函数"捕获“外部的值。

In [15]:
def wrapper(x):
    def get_sum_x(y):
        return x + y
    return get_sum_x

sum_func = wrapper(1) # 虽然 get_sum_x 根本没有 x 作为参数，他根据上下文"捕获"了上层函数的 x。
print(sum_func(2))
print(sum_func(3))

3
4


闭包可以用于固定某个函数中的一部分值，甚至也可以通过条件不同返回不同的函数来起到条件跳转的作用。

## 装饰器
使用闭包的一个最常见的实例便是装饰器。

考虑你有一堆现成的函数，现在你的程序出现了严重的 bug，需要给每个函数加上日志。
一个想法是把打日志的功能加到每一个函数里面，但是当函数很多时，这种方法并不高效（也很难进行扩展）。
装饰器的产生，就是为了对现有函数赋予新的功能。
简单来说，装饰器就是输入一个函数作为参数，返回一个函数的函数。

In [16]:
def logger(func): # 输入一个函数！
    def wrapper(*args, **kwargs): # 需要返回的函数，用 *args 和 **kwargs 表示接受任何参数，args 是列表，kwargs 是字典
        print("before calling:%s" % func.__name__) # __name__属性的值是函数的名字，这进一步体现了函数也是一个对象
        return_value = func(*args, **kwargs) # 调用原本的函数，记录返回值（回忆用*解包一个列表为位置参数，而**解包一个字典为关键词参数）
        print("after calling:%s" % func.__name__) # 再打一条日志
        return return_value # 之后返回应该返回的值
    return wrapper # 返回这个函数

@logger # 在函数定义前一行用 @装饰器 的形式附加装饰器
def add(x, y):
    return x + y

def add_no_decorator(x, y):
    return x + y
print(add(3, 5))
# 等价形式！
new_function = logger(add_no_decorator)
print(new_function(3,5))

before calling:add
after calling:add
8
before calling:add_no_decorator
after calling:add_no_decorator
8


In [17]:
def logger_custom(custom_text): # 装饰器也能有参数！
    def decorator(func): # 实际装饰器
        def wrapper(*args, **kwargs):
            print(custom_text)
            return func(*args, **kwargs)
        return wrapper
    return decorator # 返回实际装饰器，custom_text会被装饰器捕获

@logger_custom('hello')
def add(x, y):
    return x + y

print(add(3, 5))

# 等价形式！
hello_logger = logger_custom('hello') # logger_custom 返回一个装饰器!
@hello_logger # 用返回的装饰器（此时他作为闭包捕获了 custom_text）装饰函数
def add2(x, y):
    return x + y
print(add2(3, 5))

hello
8
hello
8


即使不理解也要知道装饰器很有用（尤其在第三方库里，会大量使用装饰器）

# 还有更多？
虽然其实扩展内容远不止这么多，但没时间了。推荐了解的扩展内容：
迭代器，一些重要的内置方法，多线程（这个Python不太行）、协程、多进程编程（在爬虫里尤其常用

请积极查阅文档：https://docs.python.org/zh-cn/3/index.html

不夸张地说，一切 Python 有关的东西在文档里都有很好的描述。

# 祝大家写爬虫与